In [1]:
import cv2
import numpy as  np
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Rescaling
from keras.models import Sequential

In [2]:
def crop_minAreaRect(rect, box, gray):
    mult = 1 
    W = rect[1][0]
    H = rect[1][1]

    Xs = [i[0] for i in box]
    Ys = [i[1] for i in box]
    x1 = min(Xs)
    x2 = max(Xs)
    y1 = min(Ys)
    y2 = max(Ys)

    rotated = False
    angle = rect[2]

    if angle < -45:
        angle+=90
        rotated = True

    center = (int((x1+x2)/2), int((y1+y2)/2))
    size = (int(mult*(x2-x1)),int(mult*(y2-y1)))

    M = cv2.getRotationMatrix2D((size[0]/2, size[1]/2), angle, 1.0)

    cropped = cv2.getRectSubPix(gray, size, center)    
    cropped = cv2.warpAffine(cropped, M, size)

    croppedW = W if not rotated else H 
    croppedH = H if not rotated else W

    croppedRotated = cv2.getRectSubPix(cropped, (int(croppedW*mult), int(croppedH*mult)), (size[0]/2, size[1]/2))
    
    return croppedRotated

In [3]:
input_shape = (64, 64, 3)


model = Sequential()
model.add(Rescaling((1.0 / 255), input_shape=(64, 64, 3)))
model.add(Conv2D(16, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block1'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block1_maxpool'))

model.add(Conv2D(32, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block2'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block2_maxpool'))

model.add(Conv2D(64, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block3'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block3_maxpool'))


model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 64, 64, 3)         0         
                                                                 
 block1 (Conv2D)             (None, 64, 64, 16)        448       
                                                                 
 block1_maxpool (MaxPooling2  (None, 32, 32, 16)       0         
 D)                                                              
                                                                 
 block2 (Conv2D)             (None, 32, 32, 32)        4640      
                                                                 
 block2_maxpool (MaxPooling2  (None, 16, 16, 32)       0         
 D)                                                              
                                                                 
 block3 (Conv2D)             (None, 16, 16, 64)        1

In [6]:
model = tf.keras.models.load_model("model_vgg16.h5")

In [7]:
from keras.preprocessing.image import img_to_array


In [8]:
img_crop=cv2.imread("91.jpg",1)
img_crop = cv2.resize(img_crop, (64,64), interpolation = cv2.INTER_AREA)
img_crop = img_to_array(img_crop)
img_crop = img_crop.reshape((1, img_crop.shape[0], img_crop.shape[1], img_crop.shape[2]))
Y_box_pred = model.predict(img_crop)
print(Y_box_pred)

[[1.7413183e-06]]


In [10]:
video_path='test2.mp4'
cap=cv2.VideoCapture(video_path)
backSub=cv2.createBackgroundSubtractorMOG2()
if(cap.isOpened()==False):
    print("Khong mo duoc video")
while (cap.isOpened()):
    ret, frame=cap.read()
    if ret==True:
        #Thuat toan tru nen
        fgMask=backSub.apply(frame)
        fgMask=cv2.cvtColor(fgMask, 0) 
        #Dung bo loc ma tran 5*5
        kernel = np.ones((7,7), np.uint8)
        #Lay gia tri pixel nho nhat cua kernel lam manh net
        fgMask=cv2.erode(fgMask,kernel,iterations=1)
        #Lay gia tri pixel lon nhat cua kernel lam lien net
        fgMask=cv2.dilate(fgMask,kernel,iterations=1)
        
        #tao them su lien net bang ham open
        fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel,iterations=1)
        #lay nguong 120
        _,fgMask = cv2.threshold(fgMask,120,255,cv2.THRESH_BINARY)
        #phat hien canh bang candy
        fgMask=cv2.Canny(fgMask, 20, 200)
        contours,_ = cv2.findContours(fgMask,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for i in range(len(contours)):
            rect = cv2.minAreaRect(contours[i])
            if (rect[1][0]*rect[1][1]) < 1000:
                continue
            
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            
            img_crop = crop_minAreaRect(rect, box, frame)       
            img_crop = cv2.resize(img_crop, (64,64), interpolation = cv2.INTER_AREA)
            img_crop = img_to_array(img_crop)
            img_crop = img_crop.reshape((1, img_crop.shape[0], img_crop.shape[1], img_crop.shape[2]))
            cv2.drawContours(frame,[box],0,(0,255,0),2)
        
        
            Y_box_pred = model.predict(img_crop)
            if Y_box_pred <= 0.5:
                cv2.putText(frame, "Person", (box[2][0], box[2][1]),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            if Y_box_pred > 0.5:
                cv2.putText(frame, "Non", (box[2][0], box[2][1]),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

        cv2.imshow('Frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
        
cv2.destroyAllWindows()    
            